In [1]:
# imports
import tensorflow.keras as keras


2023-06-29 10:54:58.618360: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-29 10:54:59.156252: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-29 10:54:59.156335: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# Task 0
def vanilla_autoencoder(input_dims, hidden_layers, latent_dims):
    """
    Creates a vanilla autoencoder

    Args:
        input_dims - integer containing the dimensions of the model input
        hidden_layers - list containing the number of nodes for each hidden
            layer in the encoder, respectively
        latent_dims - integer containing the dimensions of the latent space
            representation

    Returns:
        encoder - the encoder model
        decoder - the decoder model
        auto - the full autoencoder model
    """

    # First, build the encoder
    input_img = keras.Input(shape=(input_dims,))
    encoded = input_img
    for layer in hidden_layers:
        encoded = keras.layers.Dense(layer, activation='relu')(encoded)
    encoded = keras.layers.Dense(latent_dims, activation='relu')(encoded)
    encoder = keras.Model(input_img, encoded, name='encoder')

    # Now, build the decoder
    decoder_img = keras.Input(shape=(latent_dims,))
    decoded = decoder_img
    for layer in hidden_layers[::-1]:
        decoded = keras.layers.Dense(layer, activation='relu')(decoded)
    decoded = keras.layers.Dense(input_dims, activation='sigmoid')(decoded)
    decoder = keras.Model(decoder_img, decoded, name='decoder')

    # Let's put it all together
    encoded_output = encoder(input_img)
    decoded_output = decoder(encoded_output)
    autoencoder = keras.Model(input_img,
                              decoded_output,
                              name='autoencoder')

    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return encoder, decoder, autoencoder


In [ ]:
# 0-main
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.keras.datasets import mnist

# autoencoder = __import__('0-vanilla').autoencoder

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
np.random.seed(0)
tf.set_random_seed(0)
encoder, decoder, auto = vanilla_autoencoder(784, [128, 64], 32)
auto.fit(x_train, x_train, epochs=50,batch_size=256, shuffle=True,
                validation_data=(x_test, x_test))
encoded = encoder.predict(x_test[:10])
print(np.mean(encoded))
reconstructed = decoder.predict(encoded)

for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    ax.axis('off')
    plt.imshow(x_test[i].reshape((28, 28)))
    ax = plt.subplot(2, 10, i + 11)
    ax.axis('off')
    plt.imshow(reconstructed[i].reshape((28, 28)))
plt.show()

In [ ]:
# Task 1 - Sparse autoencoder
# Add a lambtha, to be used as the L1 paramater for regularization

def sparse_autoencoder(input_dims, hidden_layers, latent_dims, lambtha):
    """
    Creates a sparse autoencoder

        Args:
        input_dims - integer containing the dimensions of the model input
        hidden_layers - list containing the number of nodes for each hidden
            layer in the encoder, respectively
        latent_dims - integer containing the dimensions of the latent space
            representation
        lambtha - regularization paramater used for L1 regularization on the
            encoded output

    Returns:
        encoder - the encoder model
        decoder - the decoder model
        auto - the full autoencoder model
    """

    # Create reg variable to keep line length short
    reg = keras.regularizers.l1(lambtha)

    # First, build the encoder
    input_img = keras.Input(shape=(input_dims,))
    encoded = input_img
    for layer in hidden_layers:
        encoded = keras.layers.Dense(layer,
                                     activation='relu'
                                     )(encoded)
    encoded = keras.layers.Dense(latent_dims,
                                 activation='relu',
                                 activity_regularizer=reg
                                 )(encoded)
    encoder = keras.Model(input_img, encoded, name='encoder')

    # Now, build the decoder
    decoder_img = keras.Input(shape=(latent_dims,))
    decoded = decoder_img
    for layer in hidden_layers[::-1]:
        decoded = keras.layers.Dense(layer, activation='relu')(decoded)
    decoded = keras.layers.Dense(input_dims, activation='sigmoid')(decoded)
    decoder = keras.Model(decoder_img, decoded, name='decoder')

    # Let's put it all together
    encoded_output = encoder(input_img)
    decoded_output = decoder(encoded_output)
    autoencoder = keras.Model(input_img,
                              decoded_output,
                              name='autoencoder')

    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return encoder, decoder, autoencoder

In [ ]:
# 1-main

import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.keras.datasets import mnist

(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
np.random.seed(0)
tf.set_random_seed(0)
encoder, decoder, auto = sparse_autoencoder(784, [128, 64], 32, 10e-6)
auto.fit(x_train, x_train, epochs=100,batch_size=256, shuffle=True,
                validation_data=(x_test, x_test))
encoded = encoder.predict(x_test[:10])
print(np.mean(encoded))
reconstructed = decoder.predict(encoded)

for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    ax.axis('off')
    plt.imshow(x_test[i].reshape((28, 28)))
    ax = plt.subplot(2, 10, i + 11)
    ax.axis('off')
    plt.imshow(reconstructed[i].reshape((28, 28)))
plt.show()

In [ ]:
# Convolutional Autoencoder
def conv_autoencoder(input_dims, filters, latent_dims):
    """
    Create a convolutional autoencoder

    Each convolution in the encoder and decoder should use a kernel size of
        (3, 3) with same padding and relu activation followed by max pooling
        size of (2, 2)

    Args:
        input_dims - tuple of integers containing the dimensions of the input
        filters - list containing the number of filters for each convolutional
            layer in the encoder, respectively
        The filters should be reversed for the decoder
        latent_dims - tuple of integers containing the dimensions of the latent
            space representation

    Returns:
        encoder - the encoder model
        decoder - the decoder model
        auto - the full autoencoder model
    """
    # i = 0
    # Build the encoder
    input_img = keras.Input(shape=input_dims)
    encoded = input_img
    for filter in filters:
        encoded = keras.layers.Conv2D(filter,
                                      (3, 3),
                                      activation='relu',
                                      padding='same')(encoded)
        encoded = keras.layers.MaxPooling2D((2, 2),
                                            padding='same')(encoded)
    encoder = keras.Model(input_img, encoded, name='encoder')

    # Build the decoder
    decoder_img = keras.Input(shape=latent_dims)
    decoded = decoder_img
    # print(filters[0])
    # print(filters[1])
    # print(filters[2])
    # Loop through filters backwards
    # for filter in filters[1::-1]:
    #     print("loop: " + str(i))
    #     print(filter)
    #     i += 1
    #     decoded = keras.layers.Conv2D(filter - 1,
    #                                   (3, 3),
    #                                   activation='relu',
    #                                   padding='same')(decoded)
    #     decoded = keras.layers.UpSampling2D((2, 2))(decoded)
    # print(filters[0])
    # Going to try to manually loop
    decoded = keras.layers.Conv2D(filters[2],
                                  (3, 3),
                                  activation='relu',
                                  padding='same')(decoded)
    decoded = keras.layers.UpSampling2D((2, 2))(decoded)
    
    decoded = keras.layers.Conv2D(filters[1],
                                  (3, 3),
                                  activation='relu',
                                  padding='same')(decoded)
    decoded = keras.layers.UpSampling2D((2, 2))(decoded)

    decoded = keras.layers.Conv2D(filters[0],
                                  (3, 3),
                                  activation='relu',
                                  padding='valid')(decoded)
    decoded = keras.layers.UpSampling2D((2, 2))(decoded)
    # print("Exited loop")
    decoded = keras.layers.Conv2D(1,
                                  (3, 3),
                                  activation='sigmoid',
                                  padding='same')(decoded)
    decoder = keras.Model(decoder_img,
                          decoded,
                          name='decoder')

    # Bring it all together
    encoded_output = encoder(input_img)
    decoded_output = decoder(encoded_output)
    autoencoder = keras.Model(input_img,
                              decoded_output,
                              name='autoencoder')

    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return encoder, decoder, autoencoder

In [ ]:
# 2-main
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.keras.datasets import mnist


(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)
print(x_train.shape)
print(x_test.shape)
np.random.seed(0)
tf.set_random_seed(0)
encoder, decoder, auto = conv_autoencoder((28, 28, 1), [16, 8, 8], (4, 4, 8))
auto.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True,
                validation_data=(x_test, x_test))
encoded = encoder.predict(x_test[:10])
print(np.mean(encoded))
reconstructed = decoder.predict(encoded)[:,:,:,0]

for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    ax.axis('off')
    plt.imshow(x_test[i,:,:,0])
    ax = plt.subplot(2, 10, i + 11)
    ax.axis('off')
    plt.imshow(reconstructed[i])
plt.show()

In [27]:
# Task 3 - Variational Autoencoder

def var_autoencoder(input_dims, hidden_layers, latent_dims):
    """
    Create a variational autoencoder

    Args:
        input_dims - integer containing the dimension of the input
        hidden_layers - list containing the number of nodes for each hidden
            layer in the encoder, respectively
        The filters should be reversed for the decoder
        latent_dims - integer containing the dimension of the latent
            space representation

    Returns:
        encoder - the encoder model, which should output the latent
            representation, the mena, and the log variance, respectively
        decoder - the decoder model
        auto - the full autoencoder model
    """
    def sampling(args):
        """
        Method to sample new, similar points from the latent space
        """
        z_mean, z_log_sigma = args
        shape = keras.backend.shape(z_mean)[0], latent_dims
        epsilon = keras.backend.random_normal(shape=shape, mean=0, stddev=0.1)

        return z_mean + keras.backend.exp(z_log_sigma) * epsilon

    # First, build the encoder
    input_img = keras.Input(shape=(input_dims,))

    h = input_img
    for layer in hidden_layers:
        h = keras.layers.Dense(layer, activation='relu')(h)

    # h = keras.layers.Dense(latent_dims)(h)
    z_mean = keras.layers.Dense(latent_dims)(h)
    z_log_sigma = keras.layers.Dense(latent_dims)(h)

    z = keras.layers.Lambda(sampling)([z_mean, z_log_sigma])

    encoder = keras.Model(input_img, [z_mean, z_log_sigma, z], name='encoder')

    # I think I'm below here. Seems to be the encoder that's failing

    # Now the decoder
    latent_inputs = keras.Input(shape=(latent_dims,))

    x = latent_inputs
    for layer in hidden_layers[1::-1]:
        x = keras.layers.Dense(layer, activation='relu')(x)
    outputs = keras.layers.Dense(input_dims, activation='sigmoid')(x)
    decoder = keras.Model(latent_inputs, outputs, name='decoder')

    # instantiate the VAE model
    outputs = decoder(encoder(input_img)[2])
    vae = keras.Model(input_img, outputs, name='vae_mlp')

    reconstruction_loss = keras.losses.binary_crossentropy(input_img,
                                                           outputs)
    reconstruction_loss *= input_dims
    kl_loss = 1 + z_log_sigma - keras.backend.square(z_mean)
    kl_loss -= keras.backend.exp(z_log_sigma)
    kl_loss = keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = keras.backend.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return encoder, decoder, vae


In [2]:
import tensorflow.keras as keras
layers = keras.layers
Model = keras.Model

def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

def var_autoencoder(input_dims, hidden_layers, latent_dims):
    # Encoder
    encoder_inputs = layers.Input(shape=(input_dims,))
    x = encoder_inputs
    for units in hidden_layers:
        x = layers.Dense(units, activation='relu')(x)
    z_mean = layers.Dense(latent_dims)(x)
    z_log_var = layers.Dense(latent_dims)(x)
    z = layers.Lambda(sampling)([z_mean, z_log_var])
    encoder = Model(encoder_inputs, [z, z_mean, z_log_var])

    # Decoder
    latent_inputs = layers.Input(shape=(latent_dims,))
    x = latent_inputs
    for units in reversed(hidden_layers):
        x = layers.Dense(units, activation='relu')(x)
    decoder_outputs = layers.Dense(input_dims, activation='sigmoid')(x)
    decoder = Model(latent_inputs, decoder_outputs)

    # Full Autoencoder
    autoencoder_outputs = decoder(encoder(encoder_inputs)[0])
    autoencoder = Model(encoder_inputs, autoencoder_outputs)

    # Compile the autoencoder
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return encoder, decoder, autoencoder

In [ ]:
# 3-main
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.keras.datasets import mnist


(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))
np.random.seed(0)
tf.set_random_seed(0)
encoder, decoder, auto = var_autoencoder(784, [512], 2)
auto.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True,
                validation_data=(x_test, x_test))
encoded, mu, log_sig = encoder.predict(x_test[:10])
print(mu)
print(np.exp(log_sig / 2))
reconstructed = decoder.predict(encoded).reshape((-1, 28, 28))
x_test = x_test.reshape((-1, 28, 28))

for i in range(10):
    ax = plt.subplot(2, 10, i + 1)
    ax.axis('off')
    plt.imshow(x_test[i])
    ax = plt.subplot(2, 10, i + 11)
    ax.axis('off')
    plt.imshow(reconstructed[i])
plt.show()


l1 = np.linspace(-3, 3, 25)
l2 = np.linspace(-3, 3, 25)
L = np.stack(np.meshgrid(l1, l2, sparse=False, indexing='ij'), axis=2)
G = decoder.predict(L.reshape((-1, 2)), batch_size=125)

for i in range(25*25):
    ax = plt.subplot(25, 25, i + 1)
    ax.axis('off')
    plt.imshow(G[i].reshape((28, 28)))
plt.show()

In [3]:
# Checker Main File
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.keras.datasets import mnist

# autoencoder = __import__('3-variational').autoencoder

np.random.seed(0)
tf.set_random_seed(0)
encoder, decoder, auto = var_autoencoder(784, [512, 256], 2)
if len(auto.layers) == 3:
    print(auto.layers[0].input_shape == [(None, 784)])
    print(auto.layers[1] is encoder)
    print(auto.layers[2] is decoder)

with open('1-test', 'w+') as f:
    x_test = np.load("/home/bsbanotto/holbertonschool-machine_learning/supervised_learning/data/MNIST.npz")["X_test"]
    x_test = x_test[:256].reshape((-1, 784))
    f.write(np.format_float_scientific(auto.evaluate(x_test, x_test, verbose=False), precision=6) + '\n')
    f.write(auto.optimizer.__class__.__name__ + '\n')

with open('2-test', 'w+') as f:
    try:
        f.write(encoder.layers[0].__class__.__name__ + '\n')
        f.write(str(encoder.layers[0].input_shape) + '\n')
    except:
        f.write('FAIL\n')
    for layer in encoder.layers[1:]:
        try:
            f.write(layer.__class__.__name__ + '\n')
            if layer.__class__.__name__ == 'Dense':
                if layer.activation is not None:
                    f.write(layer.activation.__name__ + '\n')
                f.write(str(layer.input_shape) + '\n')
                f.write(str(layer.output_shape) + '\n')
            elif layer.__class__.__name__ == 'Lambda':
                assert(len(layer.input) == 2)
                assert(encoder.layers[-3].output in layer.input)
                assert(encoder.layers[-2].output in layer.input)
                f.write(str(layer.input_shape) + '\n')
                f.write(str(layer.output_shape) + '\n')
        except:
            f.write('FAIL\n')

with open('3-test', 'w+') as f:
    try:
        f.write(decoder.layers[0].__class__.__name__ + '\n')
        f.write(str(decoder.layers[0].input_shape) + '\n')
    except:
        f.write('FAIL\n')
    for layer in decoder.layers[1:]:
        try:
            f.write(layer.__class__.__name__ + '\n')
            if layer.__class__.__name__ == 'Dense' and layer.activation != None:
                f.write(layer.activation.__name__ + '\n')
            f.write(str(layer.input_shape) + '\n')
            f.write(str(layer.output_shape) + '\n')
        except:
            f.write('FAIL\n')

2023-06-29 13:30:44.341936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-29 13:30:44.342146: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-29 13:30:44.342214: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (BensLaptop): /proc/driver/nvidia/version does not exist
2023-06-29 13:30:44.345695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


True
True
True
